In [27]:
import pulp
import numpy as np
import matplotlib_inline as plt

In [28]:
prob = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)

In [29]:
nodes = ['a','b','c','d','r']
edges_with_costs ={
    ('a','b'):3,('c','a'):1,('r','a'):9,('b','c'):8,
    ('b','r'):4,('d','b'):6,('r','c'):2,
    ('c','d'):7,('d','c'):5
}

In [30]:
import pulp
x = pulp.LpVariable.dicts('x',(nodes,nodes),0,1,cat=pulp.LpInteger)

In [31]:
prob +=pulp.lpSum(edges_with_costs[u,v] * x[u][v] for u, v in edges_with_costs)

In [32]:
for v in nodes:
    if v != 'r':
        prob +=pulp.lpSum(x[u][v] for u in nodes if (u,v) in edges_with_costs) == 1
        #prob +=pulp.lpSum(x[u,v] for u in nodes if (v,u) in edges_with_costs) == 1

In [33]:
prob += pulp.lpSum(x[u]['r'] for u in nodes if (u,'r') in edges_with_costs) == 0
prob += pulp.lpSum(x['r'][v] for v in nodes if ('r',v) in edges_with_costs)>= 1

In [34]:
from pulp import *
n = len(nodes)
subsets = [tuple(c) for c in allcombinations(nodes,n-1)]
subset = []
for s in subsets:
    if 'r' in s:
        subset.append(s)
subset

[('r',),
 ('a', 'r'),
 ('b', 'r'),
 ('c', 'r'),
 ('d', 'r'),
 ('a', 'b', 'r'),
 ('a', 'c', 'r'),
 ('a', 'd', 'r'),
 ('b', 'c', 'r'),
 ('b', 'd', 'r'),
 ('c', 'd', 'r'),
 ('a', 'b', 'c', 'r'),
 ('a', 'b', 'd', 'r'),
 ('a', 'c', 'd', 'r'),
 ('b', 'c', 'd', 'r')]

In [35]:
for s in subset:
    sc = list(set(nodes)-set(s)) # complement of subset s
    summation = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation += x[u][v]
    prob += summation >= 1  


In [36]:
prob.solve()
print ("Status:", LpStatus[prob.status])

Status: Optimal


In [39]:
print ("Optimal Solution")
resultin = []
for i in nodes:
	for j in nodes:
		if x[i][j].value() == 1:
			resultin.append((i,j))
resultin

Optimal Solution


[('a', 'b'), ('c', 'a'), ('c', 'd'), ('r', 'c')]

In [40]:
costin = 0.0
for e in resultin:
    costin+= edges_with_costs[e]
costin

13.0